In [1]:
import os

os.chdir('../')

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path : Path
    base_model_path : Path
    training_data : Path
    params_epochs : int
    params_batch_size : int
    params_loss_function : str
    params_optimizer : str

In [3]:
from electrycityconsumption.constants.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from electrycityconsumption.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = Path(self.data_ingestion.local_data_path)
       

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            base_model_path=Path(prepare_base_model.base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_loss_function=params.LOSS_FUNCTION,
            params_optimizer=params.OPTIMIZER

        )

        return training_config



In [4]:
import tensorflow as tf
from pathlib import Path
import time

import pandas as pd

class Training:
    def __init__(self, config : TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.base_model_path
        )

    def save_model(self, path: Path, model: tf.keras.Model):
        model.save(path)

    def load_training_data(training_data_path):
        try:
            # Load your training data from the provided CSV file
            data = pd.read_csv(training_data_path)

            # Assuming your data has columns for features and a target variable 'Usage_kWh'
            # Replace with the actual column names from your data
            X = data.drop(columns=['Usage_kWh'])  # Exclude the target variable
            y = data['Usage_kWh']  # Target variable

          

            return X,y
        except Exception as e:
            print(f"Error loading training data: {str(e)}")
            return None, None

    
    


    def train(self):
        self.create_lstm_model()
        #Load your training data
        X_train, y_train = self.load_training_data(self.config.training_data)

        # Split your data into training and validation sets
        X_train, y_train, X_valid, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

        # Train your model
        history = self.model.fit(X_train, y_train, epochs=self.config.params_epochs, validation_data=(X_valid, y_valid))

        # Save your trained model
        self.save_model(self.config.trained_model_path, self.model)

ModuleNotFoundError: No module named 'sklearn'

In [5]:
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'